### 1. Import Libraries and Load the Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import warnings
import os
warnings.filterwarnings('ignore')

# Set up paths
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

# Create directories if they don't exist
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

file_path = RAW_DATA_DIR / "complaints.csv"

# Load the data (robust to large files)
print("Loading data...")
# Quick file size check
try:
    file_size_mb = os.path.getsize(file_path) / (1024**2)
except Exception as e:
    print('Could not determine file size:', e)

# Read a small sample first to inspect columns and dtypes
try:
    sample = pd.read_csv(file_path, nrows=1000, low_memory=False)
except Exception as e:
    print('Error reading sample:', e)

# Try a normal read but fall back to chunked reading on failure or memory issues
try:
    df = pd.read_csv(file_path, low_memory=False)
except Exception as e:
    print('Full read failed with error:', type(e).__name__, e)
    print('Falling back to chunked read (streaming).')
    chunks = []
    chunksize = 100000
    try:
        for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize, low_memory=False, iterator=True)):
            print(f'  Read chunk {i+1} with {len(chunk):,} rows')
            chunks.append(chunk)
    except Exception as e2:
        print('Chunked read failed:', type(e2).__name__, e2)
        raise
    df = pd.concat(chunks, ignore_index=True) if chunks else pd.DataFrame()

# Display first few rows (or sample if full file not loaded)
print("\nFirst 5 rows:")
display(df.head() if not df.empty else (sample.head() if 'sample' in locals() else None))

### 2. Initial EDA Analysis

In [ ]:
# 1. Basic information
print("=== BASIC INFORMATION ===")
print(f"Total records: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print("\nColumn names and data types:")
print(df.dtypes)
print("\nMissing values per column:")
print(df.isnull().sum().sort_values(ascending=False).head(20))

# 2. Analyze complaint distribution across products
print("\n=== PRODUCT DISTRIBUTION ===")
product_counts = df['Product'].value_counts()
print(f"Number of unique products: {len(product_counts)}")
print("\nTop 20 products by complaint count:")
print(product_counts.head(20))

# Visualize product distribution
plt.figure(figsize=(12, 6))
top_products = product_counts.head(15)
bars = plt.barh(top_products.index, top_products.values)
plt.xlabel('Number of Complaints')
plt.title('Top 15 Products by Complaint Count')
plt.gca().invert_yaxis()  # Highest at top
for bar in bars:
    width = bar.get_width()
    plt.text(width + 100, bar.get_y() + bar.get_height()/2, 
             f'{int(width):,}', ha='left', va='center')
plt.tight_layout()
plt.savefig(PROCESSED_DATA_DIR / 'product_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Analyze Consumer complaint narrative field
print("\n=== CONSUMER COMPLAINT NARRATIVE ANALYSIS ===")

# Check for narratives
has_narrative = df['Consumer complaint narrative'].notna()
narrative_stats = {
    'With Narrative': has_narrative.sum(),
    'Without Narrative': (~has_narrative).sum(),
    'Percentage with Narrative': (has_narrative.sum() / len(df) * 100)
}

for key, value in narrative_stats.items():
    print(f"{key}: {value:,.0f}" if 'Percentage' not in key else f"{key}: {value:.1f}%")

# Visualize narrative presence
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart
ax[0].pie([narrative_stats['With Narrative'], narrative_stats['Without Narrative']],
          labels=['With Narrative', 'Without Narrative'],
          autopct='%1.1f%%', startangle=90)
ax[0].set_title('Presence of Consumer Complaint Narrative')

# Bar chart
ax[1].bar(['With Narrative', 'Without Narrative'], 
          [narrative_stats['With Narrative'], narrative_stats['Without Narrative']])
ax[1].set_ylabel('Count')
ax[1].set_title('Complaints with/without Narrative')
for i, v in enumerate([narrative_stats['With Narrative'], narrative_stats['Without Narrative']]):
    ax[1].text(i, v + 1000, f'{v:,}', ha='center', va='bottom')
plt.tight_layout()
plt.savefig(PROCESSED_DATA_DIR / 'narrative_presence.png', dpi=300, bbox_inches='tight')
plt.show()

# 4. Analyze narrative length
print("\n=== NARRATIVE LENGTH ANALYSIS ===")

# Calculate word count for narratives that exist
df['narrative_word_count'] = df['Consumer complaint narrative'].apply(
    lambda x: len(str(x).split()) if pd.notna(x) else 0
)

# Filter only those with narratives
narratives_df = df[df['narrative_word_count'] > 0]

print(f"Number of narratives: {len(narratives_df):,}")
print(f"Average word count: {narratives_df['narrative_word_count'].mean():.1f}")
print(f"Median word count: {narratives_df['narrative_word_count'].median():.0f}")
print(f"Min word count: {narratives_df['narrative_word_count'].min()}")
print(f"Max word count: {narratives_df['narrative_word_count'].max()}")

# Visualize narrative length distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Histogram
axes[0].hist(narratives_df['narrative_word_count'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Word Count')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Narrative Length (Word Count)')
axes[0].axvline(narratives_df['narrative_word_count'].mean(), color='red', linestyle='--', 
                label=f'Mean: {narratives_df["narrative_word_count"].mean():.1f}')
axes[0].axvline(narratives_df['narrative_word_count'].median(), color='green', linestyle='--', 
                label=f'Median: {narratives_df["narrative_word_count"].median():.0f}')
axes[0].legend()

# Box plot
axes[1].boxplot(narratives_df['narrative_word_count'])
axes[1].set_ylabel('Word Count')
axes[1].set_title('Box Plot of Narrative Length')
axes[1].set_xticklabels([''])

# Log scale histogram (to see distribution better)
axes[2].hist(narratives_df['narrative_word_count'], bins=50, edgecolor='black', alpha=0.7, log=True)
axes[2].set_xlabel('Word Count (log scale)')
axes[2].set_ylabel('Frequency (log)')
axes[2].set_title('Narrative Length (Log Scale)')

plt.tight_layout()
plt.savefig(PROCESSED_DATA_DIR / 'narrative_length_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# 5. Check for very short or very long narratives
short_threshold = 10  # Less than 10 words
long_threshold = 1000  # More than 1000 words

short_narratives = narratives_df[narratives_df['narrative_word_count'] < short_threshold]
long_narratives = narratives_df[narratives_df['narrative_word_count'] > long_threshold]

print(f"\nNarratives with less than {short_threshold} words: {len(short_narratives):,} ({len(short_narratives)/len(narratives_df)*100:.2f}%)")
print(f"Narratives with more than {long_threshold} words: {len(long_narratives):,} ({len(long_narratives)/len(narratives_df)*100:.2f}%)")

# 6. Analyze by date if available
if 'Date received' in df.columns:
    print("\n=== TEMPORAL ANALYSIS ===")
    df['Date received'] = pd.to_datetime(df['Date received'], errors='coerce')
    df['Year'] = df['Date received'].dt.year
    df['Month'] = df['Date received'].dt.month
    
    yearly_counts = df['Year'].value_counts().sort_index()
    
    plt.figure(figsize=(10, 5))
    plt.plot(yearly_counts.index, yearly_counts.values, marker='o')
    plt.xlabel('Year')
    plt.ylabel('Number of Complaints')
    plt.title('Complaints Over Time')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(PROCESSED_DATA_DIR / 'complaints_over_time.png', dpi=300, bbox_inches='tight')
    plt.show()

### Filter and Clean Data

In [ ]:
# 1. Filter to only the four required product categories
print("=== FILTERING TO REQUIRED PRODUCTS ===")

# Define the products we need (matching CFPB dataset categories)
required_products = [
    'Credit card',
    'Credit card or prepaid card',  # Include variations
    'Personal loan',
    'Payday loan',  # Sometimes categorized separately
    'Student loan',  # Sometimes categorized separately
    'Vehicle loan or lease',  # Sometimes categorized separately
    'Savings account',
    'Checking or savings account',  # Include variations
    'Money transfer',
    'Virtual currency'  # Sometimes related to money transfers
]

# Standardize product names (map variations to our 4 main categories)
product_mapping = {
    # Credit Cards
    'Credit card': 'Credit card',
    'Credit card or prepaid card': 'Credit card',
    'Prepaid card': 'Credit card',
    
    # Personal Loans
    'Personal loan': 'Personal loan',
    'Payday loan': 'Personal loan',
    'Student loan': 'Personal loan',
    'Vehicle loan or lease': 'Personal loan',
    'Consumer Loan': 'Personal loan',
    
    # Savings Accounts
    'Savings account': 'Savings account',
    'Checking or savings account': 'Savings account',
    'Bank account or service': 'Savings account',
    
    # Money Transfers
    'Money transfer': 'Money transfer',
    'Virtual currency': 'Money transfer',
    'Money transfers': 'Money transfer'
}

# Create a standardized product column
df['Product_standardized'] = df['Product'].map(product_mapping)

# Filter to only our 4 main categories
filtered_df = df[df['Product_standardized'].isin(['Credit card', 'Personal loan', 
                                                   'Savings account', 'Money transfer'])]

print(f"Original dataset size: {len(df):,}")
print(f"Filtered dataset size: {len(filtered_df):,}")
print(f"Percentage retained: {len(filtered_df)/len(df)*100:.1f}%")

# Show distribution after filtering
print("\nDistribution after filtering:")
print(filtered_df['Product_standardized'].value_counts())

# 2. Remove records with empty narratives
print("\n=== REMOVING EMPTY NARRATIVES ===")
original_count = len(filtered_df)
filtered_df = filtered_df[filtered_df['Consumer complaint narrative'].notna() & 
                          (filtered_df['Consumer complaint narrative'].str.strip() != '')]
print(f"Removed {original_count - len(filtered_df):,} records with empty narratives")
print(f"Final dataset size: {len(filtered_df):,}")

# 3. Text cleaning function
def clean_narrative(text):
    """
    Clean consumer complaint narratives for better embedding quality.
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove common boilerplate phrases
    boilerplate_phrases = [
        r'i am writing to file a complaint',
        r'this is a complaint regarding',
        r'to whom it may concern',
        r'dear sir/madam',
        r'i would like to complain',
        r'i am writing to complain',
        r'complaint number',
        r'case number',
        r'reference number',
        r'my complaint is about',
        r'my complaint concerns',
        r'i am writing this complaint',
        r'this complaint is about',
        r'i am submitting this complaint',
        r'please find my complaint below',
        r'i am writing to report',
        r'i wish to file a complaint',
        r'this letter is a formal complaint'
    ]
    
    for phrase in boilerplate_phrases:
        text = text.replace(phrase, '')
    
    # Remove special characters but keep basic punctuation
    import re
    text = re.sub(r'[^\w\s.,!?]', ' ', text)  # Keep basic punctuation
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    return text

# 4. Apply cleaning to narratives
print("\n=== CLEANING TEXT NARRATIVES ===")
print("Applying text cleaning...")

# Create a copy of the narrative column
filtered_df['Consumer complaint narrative_original'] = filtered_df['Consumer complaint narrative']
filtered_df['Consumer complaint narrative_cleaned'] = filtered_df['Consumer complaint narrative'].apply(clean_narrative)

# Check cleaning effect
sample_idx = 0
print("\nSample cleaning (first complaint):")
print("Original:")
print(filtered_df.iloc[sample_idx]['Consumer complaint narrative_original'][:500])
print("\nCleaned:")
print(filtered_df.iloc[sample_idx]['Consumer complaint narrative_cleaned'][:500])

# 5. Calculate cleaned length statistics
filtered_df['cleaned_word_count'] = filtered_df['Consumer complaint narrative_cleaned'].apply(
    lambda x: len(str(x).split())
)

print("\n=== CLEANED NARRATIVE STATISTICS ===")
print(f"Average word count (cleaned): {filtered_df['cleaned_word_count'].mean():.1f}")
print(f"Median word count (cleaned): {filtered_df['cleaned_word_count'].median():.0f}")
print(f"Total words in dataset: {filtered_df['cleaned_word_count'].sum():,}")

# 6. Save the cleaned and filtered dataset
print("\n=== SAVING CLEANED DATASET ===")
output_path = PROCESSED_DATA_DIR / "filtered_complaints.csv"
filtered_df.to_csv(output_path, index=False)
print(f"Saved cleaned dataset to: {output_path}")
print(f"File size: {output_path.stat().st_size / (1024**2):.2f} MB")

# 7. Summary statistics for report
print("\n=== FINAL SUMMARY ===")
print(f"Total complaints in final dataset: {len(filtered_df):,}")
print("\nProduct distribution:")
for product, count in filtered_df['Product_standardized'].value_counts().items():
    percentage = count / len(filtered_df) * 100
    print(f"  - {product}: {count:,} ({percentage:.1f}%)")

print(f"\nAverage narrative length: {filtered_df['cleaned_word_count'].mean():.1f} words")
print(f"Median narrative length: {filtered_df['cleaned_word_count'].median():.0f} words")
print(f"Shortest narrative: {filtered_df['cleaned_word_count'].min()} words")
print(f"Longest narrative: {filtered_df['cleaned_word_count'].max()} words")

# 8. Visualize final distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Product distribution
product_counts = filtered_df['Product_standardized'].value_counts()
axes[0].bar(product_counts.index, product_counts.values, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
axes[0].set_xlabel('Product Category')
axes[0].set_ylabel('Number of Complaints')
axes[0].set_title('Complaint Distribution by Product Category')
for i, v in enumerate(product_counts.values):
    axes[0].text(i, v + 100, f'{v:,}', ha='center', va='bottom')

# Narrative length by product
product_groups = []
for product in filtered_df['Product_standardized'].unique():
    product_data = filtered_df[filtered_df['Product_standardized'] == product]
    product_groups.append(product_data['cleaned_word_count'].values)

axes[1].boxplot(product_groups, labels=product_counts.index)
axes[1].set_xlabel('Product Category')
axes[1].set_ylabel('Word Count')
axes[1].set_title('Narrative Length Distribution by Product Category')
axes[1].set_yscale('log')  # Use log scale for better visualization

plt.tight_layout()
plt.savefig(PROCESSED_DATA_DIR / 'final_distribution_summary.png', dpi=300, bbox_inches='tight')
plt.show()